In [2]:
from skimage.io import imread, imsave
from skimage.color import rgb2gray
import cv2

import numpy as np

from google.colab import drive
drive.mount('/content/drive')


def order_points(pts):
	# initialzie a list of coordinates that will be ordered
	# such that the first entry in the list is the top-left,
	# the second entry is the top-right, the third is the
	# bottom-right, and the fourth is the bottom-left
	rect = np.zeros((4, 2), dtype = "float32")
	# the top-left point will have the smallest sum, whereas
	# the bottom-right point will have the largest sum
	s = pts.sum(axis = 1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]
	# now, compute the difference between the points, the
	# top-right point will have the smallest difference,
	# whereas the bottom-left will have the largest difference
	diff = np.diff(pts, axis = 1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]
	# return the ordered coordinates
	return rect

def four_point_transform(image, pts):
	# obtain a consistent order of the points and unpack them
	# individually
	rect = order_points(pts)
	(tl, tr, br, bl) = rect
	# compute the width of the new image, which will be the
	# maximum distance between bottom-right and bottom-left
	# x-coordiates or the top-right and top-left x-coordinates
	widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
	widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
	maxWidth = max(int(widthA), int(widthB))
	# compute the height of the new image, which will be the
	# maximum distance between the top-right and bottom-right
	# y-coordinates or the top-left and bottom-left y-coordinates
	heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
	heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
	maxHeight = max(int(heightA), int(heightB))
	# now that we have the dimensions of the new image, construct
	# the set of destination points to obtain a "birds eye view",
	# (i.e. top-down view) of the image, again specifying points
	# in the top-left, top-right, bottom-right, and bottom-left
	# order
	dst = np.array([
		[0, 0],
		[maxWidth - 1, 0],
		[maxWidth - 1, maxHeight - 1],
		[0, maxHeight - 1]], dtype = "float32")
	# compute the perspective transform matrix and then apply it
	M = cv2.getPerspectiveTransform(rect, dst)
	warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
	# return the warped image
	return warped

lista_frame = ['1400', '2100', '5200']

####################################################################################################################
# LE A IMAGEM DE GROUND TRUTH DE ERVAS DANINHAS E A TRANSFORMA PARA UM FORMATO APROPRIADO PARA O CALCULO DAS MEDIDAS
####################################################################################################################
ervas_gt = imread('/content/drive/My Drive/PFC/marcacoes1/2875_Ervas_daninhas.png')
if len(ervas_gt.shape)==3: # se a imagem de GT de ervas daninhas tiver 3 canais
  ervas_gt = rgb2gray(ervas_gt) # converte para niveis de cinza

# Aplicar a transformada de perspectiva
pts = np.float32([[240, 0],[440,0],[0,359],[639,359]])
ervas_gt = four_point_transform(ervas_gt, pts)

# converte a imagem para booleano após a transformada de perspectiva
ervas_gt = ervas_gt/np.amax(ervas_gt)
ervas_gt = ervas_gt >= 0.5
cv2.imwrite('/content/drive/My Drive/PFC/marcacoes/ervas_gt.png',ervas_gt*255)
#####################################################################################################################

####################################################################################################################
# LE A IMAGEM DE GROUND TRUTH DE LINHAS DE PLANTIO PREDITAS PARA O CALCULO DAS MEDIDAS
####################################################################################################################
linhas_gt = imread('/content/drive/My Drive/PFC/marcacoes1/2875_Linhas_de_plantio.png')
if len(linhas_gt.shape)==3: # se a imagem de GT de linhas de plantion tiver 3 canais
  linhas_gt = rgb2gray(linhas_gt) # converte para niveis de cinza

# Aplicar a transformada de perspectiva
pts = np.float32([[240, 0],[440,0],[0,359],[639,359]])
linhas_gt = four_point_transform(linhas_gt, pts)

# converte a imagem para booleano após a transformada de perspectiva
linhas_gt = linhas_gt/np.amax(linhas_gt)
linhas_gt = linhas_gt >= 0.5
cv2.imwrite('/content/drive/My Drive/PFC/marcacoes/linhas_gt.png',linhas_gt*255)
#####################################################################################################################

####################################################################################################################
# LE A IMAGEM DE ERVAS DANINHAS PREDITAS E A TRANSFORMA PARA UM FORMATO APROPRIADO PARA O CALCULO DAS MEDIDAS
####################################################################################################################
ervas_pred = imread('/content/drive/My Drive/PFC/saida8/frame2875ErvasDaninhas.png')
ervas_pred = ervas_pred/np.amax(ervas_pred)
ervas_pred = np.array(ervas_pred, dtype='bool')
cv2.imwrite('/content/drive/My Drive/PFC/marcacoes/ervas_pred.png',ervas_pred*255)


####################################################################################################################
# LE A IMAGEM  DE LINHAS DE PLANTIO PREDITAS E A TRANSFORMA PARA UM FORMATO APROPRIADO PARA O CALCULO DAS MEDIDAS
####################################################################################################################
linhas_pred = imread('/content/drive/My Drive/PFC/saida8/frame2875LinhasCulturaCompletas.png')
linhas_pred = linhas_pred/np.amax(linhas_pred)
linhas_pred = np.array(linhas_pred, dtype='bool')
cv2.imwrite('/content/drive/My Drive/PFC/marcacoes/linhas_pred.png',linhas_pred*255)



########################################################################################################################
#MEDIDAS DE DETECCAO DE ERVAS DANINHAS
########################################################################################################################
TP_Ervas = np.sum(np.logical_and(ervas_pred, ervas_gt)) # preditos como sendo pixels de erva daninha e sao pixels de erva daninha
FP_Ervas = np.sum(np.logical_and(ervas_pred, np.logical_not(ervas_gt))) # preditos como sendo pixels de erva daninha mas nao sao pixels de erva daninha
TN_Ervas = np.sum(np.logical_and(np.logical_not(ervas_pred), np.logical_not(ervas_gt))) # preditos como não sendo pixels de ervas daninhas e realmente nao sao pixels ervas daninhas
FN_Ervas = np.sum(np.logical_and(np.logical_not(ervas_pred), ervas_gt)) # preditos como não sendo pixels de ervas daninhas, mas são pixels de ervas daninhas



precision_Ervas = float(TP_Ervas) / float(TP_Ervas + FP_Ervas)
recall_Ervas = float(TP_Ervas) / float(TP_Ervas + FN_Ervas)
F1_score_Ervas = float(2*TP_Ervas)/ float(2*TP_Ervas + FP_Ervas + FN_Ervas)
accuracy_Ervas = float(TP_Ervas + TN_Ervas) / float(TP_Ervas + TN_Ervas + FP_Ervas + FN_Ervas)


########################################################################################################################
#MEDIDAS DE DETECCAO DE LINHAS DE PLANTIO
########################################################################################################################
TP_Linhas = np.sum(np.logical_and(linhas_pred, linhas_gt)) # preditos como pixels de linha de plantio e sao pixels linha de plantio
FP_Linhas = np.sum(np.logical_and(linhas_pred, np.logical_not(linhas_gt))) # preditos como sendo pixels de linha de plantio mas nao sao pixels de linha de plantio
# TN_Linhas = np.sum(np.logical_not(linhas_pred), np.logical_not(linhas_gt)) # preditos como não sendo pixels de linhas de plantio e realmente nao sao pixels de linhas de plantio
TN_Linhas = np.sum(np.logical_and(np.logical_not(linhas_pred), np.logical_not(linhas_gt))) # preditos como não sendo pixels de linhas de plantio e realmente nao sao pixels de linhas de plantio
# FN_Linhas = np.sum(np.logical_not(linhas_pred), linhas_gt) # preditos como não sendo ervas daninhas, mas são ervas daninhas
FN_Linhas = np.sum(np.logical_and(np.logical_not(linhas_pred), linhas_gt)) # preditos como não sendo ervas daninhas, mas são ervas daninhas

precision_Linhas = float(TP_Linhas) / float(TP_Linhas + FP_Linhas)
recall_Linhas = float(TP_Linhas) / float(TP_Linhas + FN_Linhas)
F1_score_Linhas = float(2*TP_Linhas)/ float(2*TP_Linhas + FP_Linhas + FN_Linhas)
accuracy_Linhas = float(TP_Linhas + TN_Linhas) / float(TP_Linhas + TN_Linhas + FP_Linhas + FN_Linhas)



print('Resultados de deteccao de Ervas Daninhas')
print('\t TP = ', TP_Ervas)
print('\t FP = ', FP_Ervas)
print('\t TN = ', TN_Ervas)
print('\t FN = ', FN_Ervas)
print('\t precision = ', precision_Ervas*100,'%')
print('\t recall  = ', recall_Ervas*100,'%')
print('\t F1-score  = ', F1_score_Ervas*100,'%')
print('\t accuracy  = ', accuracy_Ervas*100,'%')

print('Resultados de deteccao de Linhas de Plantio')
print('\t TP = ', TP_Linhas)
print('\t FP = ', FP_Linhas)
print('\t TN = ', TN_Linhas)
print('\t FN = ', FN_Linhas)
print('\t precision = ', precision_Linhas*100,'%')
print('\t recall  = ', recall_Linhas*100,'%')
print('\t F1-score  = ', F1_score_Linhas*100,'%')
print('\t accuracy  = ', accuracy_Linhas*100,'%')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Resultados de deteccao de Ervas Daninhas
	 TP =  16
	 FP =  32
	 TN =  275245
	 FN =  116
	 precision =  33.33333333333333 %
	 recall  =  12.121212121212121 %
	 F1-score  =  17.77777777777778 %
	 accuracy  =  99.94626174162791 %
Resultados de deteccao de Linhas de Plantio
	 TP =  66794
	 FP =  13390
	 TN =  192699
	 FN =  2526
	 precision =  83.30090791180285 %
	 recall  =  96.35603000577035 %
	 F1-score  =  89.35413099315068 %
	 accuracy  =  94.2209586469578 %
